# Detailed Analysis (Bid by Bid, Card by Card)

This is a tutorial of how to do a detailed analysis of a played board.

The engine looks at the bidding and play as it originally happened, and does an analysis for every bid and every card played.

The analysis is not just a double-dummy analysis for the exact current layout (like if you would press the "GIB" button on BBO). Instead, it's an analysis over many different possible layouts (samples).

In [1]:
import os
os.chdir('..')

from nn.models import Models
from analysis import CardByCard
from util import parse_lin, display_lin
from sample import Sample
import conf

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
models = Models.from_conf(conf.load('./config/default.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('./config/default.conf'))  # Load sampling strategies

INFO:tensorflow:Restoring parameters from ..\models/gib21_model/gib21-1000000
INFO:tensorflow:Restoring parameters from ..\models/gib21_info_model/gib21_info-500000
INFO:tensorflow:Restoring parameters from ..\models/lead_model_b/lead-1000000
INFO:tensorflow:Restoring parameters from ..\models/lr3_model/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\models/lefty_model/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\models/dummy_model/dummy-920000
INFO:tensorflow:Restoring parameters from ..\models/righty_model/righty-1000000
INFO:tensorflow:Restoring parameters from ..\models/decl_model/decl-1000000


In [3]:
# we specify all the information about a board
# (it's quite tedious to enter every single thing by hand here,
# later we'll have an example of how you can give it a board played on BBO)

dealer = 'S'
vuln = [True, True]  # fist element is NS, second element is EW

hands = [
    'AJ87632.J96.753.',
    'K9.Q8542.T6.AJ74',
    'QT4.A.KJ94.KQ986',
    '5.KT73.AQ82.T532'
]

auction = ['1N', 'PASS', '4H', 'PASS', '4S', 'PASS', 'PASS', 'PASS']

play = ['C2', 'D3', 'CA', 'C6', 'D6', 'DJ', 'DQ', 'D5', 'DA', 'D7', 'DT', 'D4', 'D8', 'H6', 'H2', 'D9', 'SQ', 'S5', 'S2', 'SK', 'H4', 'HA', 'H7', 'H9', 'S4', 'C3', 'SA', 'S9', 'S3', 'C4', 'ST', 'H3', 'CK', 'C5', 'HJ', 'C7', 'C8', 'CT', 'S6', 'CJ', 'S7', 'H8', 'C9', 'D2', 'S8', 'H5', 'CQ', 'HT', 'SJ', 'HQ', 'DK', 'HK']

In [4]:
card_by_card = CardByCard(dealer, vuln, hands, auction, play, models, -1, -1, sampler, False)

In [5]:
# calling this starts the analysis
# it will go bid-by-bid and card-by-card, and will take a few moments
# possible mistakes will be annotated with ? or with ?? (if it's a bigger mistake)
# (possible mistake means that the engine does not agree with the bid/play. the engine could be wrong too :))

card_by_card.analyze()

analyzing the bidding
1N ?
PASS .
4H ?
PASS .
4S .
PASS .
PASS .
['4S', 'PASS', 'PASS']
PASS .
analyzing the play
C2


Loaded lib dds.dll


D3 ??
CA .
C6 .
D6 .
DJ .
DQ .
D5 .
DA .
D7 .
DT .
D4 .
D8 .
H6 .
H2 .
D9 .
SQ .
S5 .
S2 ??
SK .
H4 .
HA .
H7 .
H9 .
S4 .
C3 .
SA .
S9 .
S3 .
C4 .
ST .
H3 .
CK .
C5 .
HJ .
C7 .
C8 .
CT .
S6 .
CJ .
S7 .
H8 .
C9 .
D2 .
S8 .
H5 .
CQ .
HT .


In [6]:
# the engine does not agree with the 1N opening.
# indeed, it's a little offbeat with a singleton
# let's see what the engine is thinking (what would it bid instead)

card_by_card.bid_responses[0].to_dict()  # the 0 index is the first bid in the auction

{'bid': '1N',
 'candidates': [{'call': '1C', 'insta_score': 0.9971}],
 'samples': ['Kxx.8xx.AQ8x.Jxx AJ98x.QTxx.x.ATx QTx.A.KJ9x.KQ98x xx.KJ9xx.Txxx.xx',
  'J8xx.Txx.Tx.Txxx A9.KQJ8x.Q8xx.Ax QTx.A.KJ9x.KQ98x Kxxx.9xxx.Axx.Jx',
  'K8x.T.AQxx.AJTxx J9xx.KQ8xxx.8x.x QTx.A.KJ9x.KQ98x Axx.J9xxx.Txx.xx',
  'AK9x.8xxx.Qxx.Ax Jxx.KQxx.AT.Txxx QTx.A.KJ9x.KQ98x 8xx.JT9x.8xxx.Jx',
  'Jx.J8xxx.AQTx.xx K8xx.Q9xx.8x.JTx QTx.A.KJ9x.KQ98x A9xx.KTx.xxx.Axx',
  '8xx.KQxx.Q8xx.Jx 9xx.98xx.Axx.ATx QTx.A.KJ9x.KQ98x AKJx.JTxx.Tx.xxx',
  'A8x.K9xx.8xx.Txx K9xx.xx.AQxx.AJx QTx.A.KJ9x.KQ98x Jxx.QJT8xx.Tx.xx',
  '98xxx.K8x.QTx.Ax x.Q9xx.Axxx.JTxx QTx.A.KJ9x.KQ98x AKJx.JTxxx.8x.xx',
  'K98x.8xxx.Axx.xx Jx.T.QTxxx.AJTxx QTx.A.KJ9x.KQ98x Axxx.KQJ9xxx.8.x',
  'K9xx.Kxx.AQx.xxx A8xx.QT9x.8xx.AJ QTx.A.KJ9x.KQ98x Jx.J8xxx.Txx.Txx',
  'J8x.QJxx.T8xx.xx Kxxx.Kxx.Qxx.Jxx QTx.A.KJ9x.KQ98x A9x.T98xx.Ax.ATx',
  'K9.Kxx.QTx.Txxxx A8x.T98xx.Axxx.x QTx.A.KJ9x.KQ98x Jxxxx.QJxx.8x.AJ',
  'J8x.Qxxx.Tx.Axxx Ax.JTxx.8xxx.JTx QTx.A.

the engine very confidently opens `1C` and doesn't even consider `1N`

In [7]:
# what about the opening lead? let's see...

card_by_card.cards['C2'].to_dict()

{'card': 'C2',
 'candidates': [{'card': 'Cx',
   'insta_score': 0.6598,
   'expected_tricks': 10.31,
   'p_make_contract': 0.73},
  {'card': 'Hx',
   'insta_score': 0.271,
   'expected_tricks': 10.33,
   'p_make_contract': 0.74}],
 'samples': ['KT9xxx.AQ.T9.J8x QJ8x.xxx.Jxxx.9x Ax.J98x.Kxx.AKQx',
  'QTxxxx.Q9x.KJ.Kx 98.J8x.Txxxx.Q9x AKJx.Axx.9x.AJ8x',
  'KQTxxx.8x.KT.Qxx J9x.Qxx.J9xxx.8x A8x.AJ9x.xx.AKJ9',
  'AJTxxxx.Q8.Jx.Q9 x.J9xx.Txx.AJ8xx KQ98.Axx.K9xx.Kx',
  'KQ9xxxx.J8.T.Kxx 8x.9xx.J9xxx.AJ9 AJT.AQxx.Kxx.Q8x',
  'AQJ8xxx.xx.Tx.Qx Kx.Q98xx.J9x.8xx T9x.AJ.Kxxx.AKJ9',
  'KJT9xxx.Qx.x.Kxx Q8x.8x.J9xxx.QJ9 Ax.AJ9xx.KTx.A8x',
  'KJT98x.Ax.Txx.Qx xxx.9xx.Jx.KJ98x AQx.QJ8x.K9xx.Ax',
  'QJ98xx.xxx.Jx.AK xxx.J98.K9x.J9xx AKT.AQx.Txxx.Q8x',
  'Q9xxxxx.Qx.x.A8x T8.A98xx.9xx.QJ9 AKJ.Jx.KJTxx.Kxx',
  'KJT98x.A9x.xx.Jx Axxx.8xxx.J9x.Q9 Qx.QJ.KTxx.AK8xx',
  'KQ98xxx.98x.x.Kx JTx.Jxx.T9xx.AQx Ax.AQx.KJxx.J98x',
  'JT9xxxx.Q9.x.KQJ Qx.A8xx.J9xx.9xx AK8.Jxx.KTxx.A8x',
  'KT8xxxx.Jx.x.A8x Ax.98xxx.J

the engine agrees with leading a low club, but it's very close. the alternative is a low heart

In [8]:
# the engine considers dummy's discard of D3 on the first trick a big mistake.
# perhaps we should ruff instead, let's see what the engine suggests

card_by_card.cards['D3'].to_dict()

{'card': 'D3',
 'candidates': [{'card': 'S7',
   'insta_score': 0.2611,
   'expected_tricks': 11.21,
   'expected_score': 635},
  {'card': 'S2',
   'insta_score': 0.2611,
   'expected_tricks': 11.21,
   'expected_score': 635},
  {'card': 'S3',
   'insta_score': 0.2611,
   'expected_tricks': 11.21,
   'expected_score': 635},
  {'card': 'S6',
   'insta_score': 0.2611,
   'expected_tricks': 11.21,
   'expected_score': 635},
  {'card': 'S8',
   'insta_score': 0.0,
   'expected_tricks': 11.21,
   'expected_score': 635},
  {'card': 'SJ',
   'insta_score': 0.0,
   'expected_tricks': 11.06,
   'expected_score': 631},
  {'card': 'D7',
   'insta_score': 0.4917,
   'expected_tricks': 10.47,
   'expected_score': 576},
  {'card': 'D5',
   'insta_score': 0.4917,
   'expected_tricks': 10.47,
   'expected_score': 576},
  {'card': 'D3',
   'insta_score': 0.4917,
   'expected_tricks': 10.47,
   'expected_score': 576},
  {'card': 'SA',
   'insta_score': 0.0,
   'expected_tricks': 10.34,
   'expected_scor

indeed, the best play is to ruff low.

looking at the samples, we see that East has the `CA` in every sample (this is by inference because underleading an A is very unlikely)

## Analyzing a board played on BBO

In [9]:
# copy-paste from the hand records (in lin format)

lin = 'pn|You,~~M7228oka,~~M72302cm,~~M72316sq|st||md|1S4TKHJD68QC679TKA,S35H479TQKD24TAC8,S2789H3AD379JKC35,|rh||ah|Board 3|sv|e|mb|1C|an|Minor suit opening -- 3+ !C; 11-21 HCP; 12-22 total points|mb|2H|an|Aggressive weak jump overcall -- 6+ !H; 4-10 HCP |mb|d|an|Negative double -- 4+ !S; 7+ HCP; 8+ total points |mb|4H|an|The Law: 10 trump -> game support -- 4+ total points |mb|4S|an|3+ !C; 4+ !S; 16-21 HCP; 17-22 total points|mb|p|mb|p|mb|p|pg||pc|DA|pc|D3|pc|D5|pc|D6|pg||pc|C8|pc|C3|pc|CJ|pc|CA|pg||pc|S4|pc|S5|pc|S8|pc|SJ|pg||pc|H5|pc|HJ|pc|HQ|pc|HA|pg||pc|S2|pc|SA|pc|ST|pc|S3|pg||pc|H2|pc|SK|pc|H4|pc|H3|pg||pc|D8|pc|D2|pc|DJ|pc|S6|pg||pc|SQ|pc|C6|pc|H7|pc|S7|pg||pc|H8|pc|C7|pc|HK|pc|S9|pg||pc|C5|pc|C2|pc|CT|pc|HT|pg||pc|CK|pc|H9|pc|D7|pc|C4|pg||pc|DQ|pc|D4|pc|DK|pc|H6|pg||pc|D9|pc|CQ|pc|C9|pc|DT|pg||'

In [10]:
display_lin(lin)

In [11]:
board = parse_lin(lin)

In [12]:
card_by_card = CardByCard(*board, models, -1, -1, sampler, False)

In [13]:
card_by_card.analyze()

analyzing the bidding
1C .
2H .
X .
4H .
4S .
PASS .
PASS .
['4S', 'PASS', 'PASS']
PASS ?
analyzing the play
DA
D3 .
D5 .
D6 .
C8 .
C3 .
CJ .
CA .
S4 ?
S5 .
S8 .
SJ .
H5 ?
HJ .
HQ .
HA .
S2 ?
SA .
ST .
S3 .
H2 ?
SK ?
H4 .


the engine agrees with the bidding, but didn't like something in the cardplay.

playing `S4` from hand is the first mistake. apparently this play drops almost half a trick on average.

In [ ]:
card_by_card.cards['S4'].to_dict()

the opening lead of `DA` is interesting. the engine prefers the `HK` and it's the only card it considers.

In [ ]:
card_by_card.cards['DA'].to_dict()